In [ ]:
import time
import pandas as pd
import numpy as np
import joblib
from itertools import combinations
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.ensemble import HistGradientBoostingRegressor

# --- Load data (expects train.csv and test.csv in working dir) ---
try:
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
except FileNotFoundError:
    raise FileNotFoundError("train.csv or test.csv not found in working directory")

test_ids = test_df["Id"]
X = train_df.drop(["Id", "Recovery Index"], axis=1).copy()
y = train_df["Recovery Index"].values.ravel()
X_test = test_df.drop("Id", axis=1).copy()

# --- Simple preprocessing and light feature engineering ---
def fe(df):
    # map lifestyle to binary and create a few interpretable combos
    df = df.copy()
    df["Lifestyle Activities"] = df["Lifestyle Activities"].fillna("No")
    df["Lifestyle_Active"] = (df["Lifestyle Activities"] == "Yes").astype(int)
    df = df.drop("Lifestyle Activities", axis=1)
    T = df["Therapy Hours"]
    H = df["Initial Health Score"]
    S = df["Average Sleep Hours"]
    F = df["Follow-Up Sessions"]
    eps = 1e-6
    df["Therapy_Per_FollowUp"] = T / (F + eps)
    df["Sleep_Health_Product"] = S * H
    df["Combined_Treatment_Hours"] = T + F
    df["Health_Per_Sleep"] = H / (S + eps)
    df["Therapy_Health_Product"] = T * H
    return df

X = fe(X)
X_test = fe(X_test)
# align columns to avoid mismatch
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# --- Impute numeric missing values using median (fit on train) ---
imp = SimpleImputer(strategy="median")
X_imp = pd.DataFrame(imp.fit_transform(X), columns=X.columns, index=X.index)
X_test_imp = pd.DataFrame(imp.transform(X_test), columns=X_test.columns, index=X_test.index)

# --- Optionally add limited pairwise interactions among top k features (keeps model compact) ---
k_top = min(6, X_imp.shape[1])
sel = SelectKBest(f_regression, k=k_top).fit(X_imp.values, y)
top_cols = [X_imp.columns[i] for i in sel.get_support(indices=True)]
for a, b in combinations(top_cols, 2):
    name = f"{a}__x__{b}"
    X_imp[name] = X_imp[a] * X_imp[b]
    X_test_imp[name] = X_test_imp[a] * X_test_imp[b]

# --- Model pipelines (small, clear) ---
n_feats = X_imp.shape[1]
k_select = min(12, max(3, n_feats))
models = {
    "ridge": Pipeline([
        ("scaler", StandardScaler()),
        ("select", SelectKBest(score_func=f_regression, k=k_select)),
        ("ridge", RidgeCV(alphas=[0.01, 0.1, 1.0], cv=5))
    ]),
    "elasticnet": Pipeline([
        ("scaler", StandardScaler()),
        ("select", SelectKBest(score_func=f_regression, k=k_select)),
        ("en", ElasticNetCV(l1_ratio=[0.1,0.5,0.9], alphas=[0.01,0.1,1.0], cv=5, max_iter=5000))
    ]),
    "hgb": Pipeline([
        ("hgb", HistGradientBoostingRegressor(random_state=42, max_iter=800, learning_rate=0.05))
    ])
}

# --- OOF predictions (partitioning CV required) ---
cv = KFold(n_splits=10, shuffle=True, random_state=42)
oof = {}
rmse = {}
start = time.time()
for name, pipe in models.items():
    print(f"OOF {name} ...", end=" ")
    preds_oof = cross_val_predict(pipe, X_imp.values, y, cv=cv, n_jobs=-1, method="predict")
    oof[name] = preds_oof
    rmse[name] = np.sqrt(mean_squared_error(y, preds_oof))
    print(f"RMSE={rmse[name]:.4f}")
print("OOF done in", time.time()-start, "s")

# --- Stacking meta (train on OOF preds + a few top original features) ---
oof_matrix = np.vstack([oof[m] for m in oof]).T
meta_X = np.hstack([oof_matrix, X_imp[top_cols].values])
meta = RidgeCV(alphas=[0.01, 0.1, 1.0], cv=5)
meta.fit(meta_X, y)
print("Meta OOF RMSE:", np.sqrt(mean_squared_error(y, meta.predict(meta_X))))

# --- Fit base models on full data and predict test set ---
test_preds = []
for name, pipe in models.items():
    pipe.fit(X_imp.values, y)
    test_preds.append(pipe.predict(X_test_imp.values))
test_matrix = np.vstack(test_preds).T

# meta prediction on test (use same augmentation)
meta_test_X = np.hstack([test_matrix, X_test_imp[top_cols].values])
pred_stack = meta.predict(meta_test_X)

# --- Weighted fallback and small blend for stability ---
rmse_vals = np.array([rmse[m] for m in ["ridge","elasticnet","hgb"]])
w = 1.0 / (rmse_vals**2 + 1e-12)
w = w / w.sum()
pred_weighted = (test_matrix * w).sum(axis=1)

final_pred = 0.9 * pred_stack + 0.1 * pred_weighted
# keep within observed target range and round for submission
ymin, ymax = y.min(), y.max()
final_pred = np.clip(final_pred, ymin, ymax)
final_int = np.rint(final_pred).astype(int)

# --- Save submission and models ---
sub = pd.DataFrame({"Id": test_ids, "Recovery Index": final_int})
sub.to_csv("submission_ensemble_improved.csv", index=False)
joblib.dump({"models": models, "meta": meta, "weights": w, "rmse": rmse, "top_cols": top_cols},
            "ensemble_improved.joblib", compress=3)
print("Saved submission_ensemble_improved.csv and ensemble_compact.joblib")


--- Actual Data Loaded ---
Train samples: 8000, Test samples: 2000
--------------------------
